# **Keras vs PyTorch vs Caffe: Comparing the Implementation of CNN**

In this practice session, we will build the deep learning framework that will be a convolutional neural network for image classification on the same dataset in Keras, PyTorch and Caffe and we will compare the implementation in all these ways. Finally, we will see how the CNN model built in PyTorch outperforms the peers built-in Keras and Caffe.

You can read about the pros and cons of each of these frameworks in [this](https://analyticsindiamag.com/keras-vs-pytorch-vs-caffe-comparing-the-implementation-of-cnn/) article.

## **Hands-on implementation Using Keras Framework**

In the below code snippet we will import the required libraries.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras opencv-python pillow scikit-image --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

Hyperparameter:

In [3]:
batch_size = 128
num_classes = 10
epochs = 12
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train=x_train.reshape((x_train.shape[0],28,28,1))#Change (60000,28,28) to (6000,28,28,1)
print(x_train.shape)#Print the shape, the data here is not normalized
x_test=x_test.reshape((x_test.shape[0],28,28,1))
y_train=np_utils.to_categorical(y_train,num_classes=10)
y_test=np_utils.to_categorical(y_test,num_classes=10)

(60000, 28, 28, 1)


In the below code snippet we will build a deep learning model with few layers and assigning optimizers, activation functions and loss functions. 

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12


KeyboardInterrupt: ignored

## **Hands-on implementation Using Pytorch Framework.**

Installing required libraries 

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
from torch.utils.data import TensorDataset
from torchvision import transforms
from torch.autograd import Variable
from torchvision.datasets import MNIST

In the below code snippet we will load the dataset and split it into training and test sets.

In [8]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), 
]), )
test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
]), )
dataloader_args = dict(shuffle=True, batch_size=64,num_workers=1, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
test_loader = dataloader.DataLoader(test, **dataloader_args)
train_data = train.train_data
train_data = train.transform(train_data.numpy())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:64: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In the below code snippet we will build our model, and assign activation functions and optimizers.

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 548)
        self.bc1 = nn.BatchNorm1d(548) 
        self.fc2 = nn.Linear(548, 252)
        self.bc2 = nn.BatchNorm1d(252)
        self.fc3 = nn.Linear(252, 10)              
    def forward(self, x):
        a = x.view((-1, 784))
        b = self.fc1(a)
        b = self.bc1(b)
        b = F.relu(b)
        b = F.dropout(b, p=0.5) 
        b = self.fc2(b)
        b = self.bc2(b)
        b = F.relu(b)
        b = F.dropout(b, p=0.2)
        b = self.fc3(b)
        out = F.log_softmax(b)
        return out
model = Model()
model.cpu()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In the below code snippet we will train our model and while training we will assign loss function that is cross-entropy.

In [10]:
model.train()
losses = []
for epoch in range(12):
    for batch_idx, (data,data_1) in enumerate(train_loader):
        data,data_1 = Variable(data.cpu()), Variable(data_1.cpu())
        optimizer.zero_grad()
        y_pred = model(data) 
        loss = F.cross_entropy(y_pred, data_1)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()), 
                end='')         
print()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 Train Epoch: 11 [57664/60000 (96%)]	Loss: 0.349635


In [15]:
evaluate_x = Variable(test_loader.dataset.test_data.type_as(torch.FloatTensor())).cpu()
evaluate_y = Variable(test_loader.dataset.test_labels).cpu()

output = model(evaluate_x)
pred = output.data.max(1)[1]
d = pred.eq(evaluate_y.data).cpu()
accuracy = d.sum()/d.size()[0]
print('Accuracy:', accuracy) # Accuracy: 0.91

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: tensor(0.9176)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## **Hands-on implementation Using Caffe Framework.**

### **Installing Caffe**

In [2]:
!sudo apt install -y caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1 libcaffe-cpu1
  libgflags2.2 libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5
  liblmdb0 python-matplotlib-data python-tables-data python3-bs4
  python3-caffe-cpu python3-chardet python3-cycler python3-dateutil
  python3-decorator python3-gflags python3-h5py python3-html5lib
  python3-ipython python3-ipython-genutils python3-leveldb python3-lxml
  python3-matplotlib python3-networkx python3-nose python3-numexpr
  python3-numpy python3-olefile python3-pandas python3-pandas-lib
  python3-pexpect python3-pickleshare python3-pil python3-pkg-resources
  python3-prompt-toolkit python3-protobuf python3-ptyprocess python3-pygments
  py

### **Importing required libraries**

In [3]:
import os
import numpy as np
import math
import caffe
import lmdb

ModuleNotFoundError: ignored

In the below code snippet we will assign the hardware environment. 

In [ ]:
os.environ["GLOG_minloglevel"] = '2'
CAFFE_ROOT="/caffe"
os.chdir(CAFFE_ROOT) 
USE_GPU = True
if USE_GPU:
    caffe.set_device(0)
    caffe.set_mode_gpu()
else:
    caffe.set_mode_cpu()
caffe.set_random_seed(1) 
np.random.seed(24)

In the below code snippet we will define the image_generator and batch_generator which helps in data transformations.

In [ ]:
def image_generator(db_path):
    db_handle = lmdb.open(db_path, readonly=True) 
    with db_handle.begin() as db:
        cur = db.cursor() 
        for _, value in cur: 
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(value) 
            int_x = caffe.io.datum_to_array(datum) 
            x = np.asfarray(int_x, dtype=np.float32)
            yield x - 128 

def batch_generator(shape, db_path):
    gen = image_generator(db_path)
    res = np.zeros(shape) 
    while True: 
        for i in range(shape[0]):
            res[i] = next(gen) 

        yield res

In the below code snippet we will give the path of the MNIST dataset.

In [ ]:
net_path = "content/mnist/lenet_train_test.prototxt"
net = caffe.Net(net_path, caffe.TRAIN)
test_net = caffe.Net(net_path, caffe.TEST) 
net.share_with(test_net)

In the below code snippet we will train our model using MNIST dataset.

In [24]:
num_epochs = 0 
iter_num = 0 
db_path = "content/mnist/mnist_train_lmdb"
db_path_test = "content/mnist/mnist_test_lmdb"
base_lr = 0.01
gamma = 1e-4
power = 0.75

for epoch in range(num_epochs):
    print("Starting epoch {}".format(epoch))
    input_shape = net.blobs["data"].data.shape
    for batch in batch_generator(input_shape, db_path):
        iter_num += 1
        net.blobs["data"].data[...] = batch
        net.forward()
        for name, l in zip(net._layer_names, net.layers):
            for b in l.blobs:
                b.diff[...] = net.blob_loss_weights[name]
        net.backward()
        learning_rate = base_lr * math.pow(1 + gamma * iter_num, - power)
        for l in net.layers:
            for b in l.blobs:
                b.data[...] -= learning_rate * b.diff
        if iter_num % 50 == 0:
            print("Iter {}: loss={}".format(iter_num, net.blobs["loss"].data))
        if iter_num % 200 == 0:
            print("Testing network: accuracy={}, loss={}".format(*test_network(test_net, db_path_test))

SyntaxError: ignored

Using the below code snippet, we will obtain the final accuracy.

In [ ]:
print("Training finished after {} iterations".format(iter_num))
print("Final performance: accuracy={}, loss={}".format(*test_network(test_net, db_path_test)))

#**Related Articles:**

> * [PyTorch vs Keras vs Caffe](https://analyticsindiamag.com/keras-vs-pytorch-vs-caffe-comparing-the-implementation-of-cnn/)

> * [Face Emotion Recognizer](https://analyticsindiamag.com/face-emotion-recognizer-in-6-lines-of-code/)

> * [Sign Language Classification using CNN](https://analyticsindiamag.com/hands-on-guide-to-sign-language-classification-using-cnn/)

> * [Transfer Learning for multi class classification](https://analyticsindiamag.com/transfer-learning-for-multi-class-image-classification-using-deep-convolutional-neural-network/)

> * [FastAI with PyTorch for multiclass Image Classification](https://analyticsindiamag.com/fastai-with-tpu-in-pytorch-for-multiclass-image-classification/)

> * [SuffleNet V1 for Multiclass Image Classification](https://analyticsindiamag.com/complete-guide-to-shufflenet-v1-with-implementation-in-multiclass-image-classification/)

> * [Image Compression using K-Means Clustering](https://analyticsindiamag.com/beginners-guide-to-image-compression-using-k-means-clustering/)
